In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install collection
!apt-get install swig -y
!pip install numpy
!pip install matplotlib
!pip install opencv-python
!git clone https://github.com/gsethi2409/tf-pose-estimation
!pip install -r tf-pose-estimation/requirements.txt
!pip install ffmpeg-python

  Created wheel for collection: filename=collection-0.1.6-cp36-none-any.whl size=5116 sha256=a4bb02589de6ebcb002e3e3fa87e22c1b5c7316eef89c919b502dd17d9aabc99
  Stored in directory: /root/.cache/pip/wheels/9e/f2/2b/a611b0dc83b770763e7962500ef158c09dc8161d3fce6e73de
Successfully built collection
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (791 kB/s)
Selecting previously un

In [ ]:
cd tf-pose-estimation/tf_pose/pafprocess/

/tf-pose-estimation/tf_pose/pafprocess


In [ ]:
!swig -python -c++ pafprocess.i && python setup.py build_ext --inplace

running build_ext
building '_pafprocess' extension
swigging pafprocess.i to pafprocess_wrap.cpp
swig -python -c++ -o pafprocess_wrap.cpp pafprocess.i
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I. -I/usr/include/python3.6m -c pafprocess.cpp -o build/temp.linux-x86_64-3.6/pafprocess.o
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I. -I/usr/include/python3.6m -c pafprocess_wrap.cpp -o build/temp.linux-x86_64-3.6/pafprocess_wrap.o
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protecto

In [ ]:
!pip install git+https://github.com/adrianc-a/tf-slim.git@remove_contrib

  Cloning https://github.com/adrianc-a/tf-slim.git (to revision remove_contrib) to /tmp/pip-req-build-qmkys6zp
  Running command git clone -q https://github.com/adrianc-a/tf-slim.git /tmp/pip-req-build-qmkys6zp
  Running command git checkout -b remove_contrib --track origin/remove_contrib
  Switched to a new branch 'remove_contrib'
  Branch 'remove_contrib' set up to track remote branch 'remove_contrib' from 'origin'.
  Created wheel for tf-slim: filename=tf_slim-1.0-cp36-none-any.whl size=275049 sha256=ceb40eb1d6923efe03e9858c5176a9c5852859254c86c796c09b2ae5889bd3b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-hfe8_i8v/wheels/be/41/65/cd259010573e2dd0fba258f9a466059b48e38118aeec65e09f
Successfully built tf-slim


In [ ]:
cd ../..

/tf-pose-estimation


# Dev

In [ ]:
import sys
sys.path.append('.')
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from imutils import paths
import os
import ffmpeg
import imutils
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

2020-12-28 02:00:26,041 INFO Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
2020-12-28 02:00:26,068 INFO Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [ ]:
model='mobilenet_v2_large'
#show_process = True # for debug purpose, if enabled, speed for inference is dropped.
#logger.debug('initialization %s : %s' % (model, get_graph_path(model)))

resolution='432x368'
w, h = model_wh(resolution)
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

[2020-12-28 02:00:26,105] [TfPoseEstimator] [INFO] loading graph from /tf-pose-estimation/models/graph/mobilenet_v2_large/graph_opt.pb(default size=432x368)
2020-12-28 02:00:26,105 INFO loading graph from /tf-pose-estimation/models/graph/mobilenet_v2_large/graph_opt.pb(default size=432x368)


TfPoseEstimator/image
TfPoseEstimator/MobilenetV2/Conv/BatchNorm/Const
TfPoseEstimator/MobilenetV2/Conv/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv/depthwise/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv/depthwise/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv/project/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv/project/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_1/expand/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_1/expand/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_1/project/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_1/project/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_2/expand/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_2/expand/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_2/depthwise/

# Extract keypoints from Video

In [ ]:
import joblib
lst_video = list(paths.list_files('/content/drive/MyDrive/dataset/action_videos/datavatva/'))
showBG = True
model = joblib.load('/content/drive/MyDrive/dataset/xgb_30_combine.pkl') #combine

In [ ]:
lst_video = [s for s in lst_video if ".mp4" in s]

In [ ]:
#check rotation
def check_rotation(path_video_file):
     # this returns meta-data of the video file in form of a dictionary
     meta_dict = ffmpeg.probe(path_video_file)

     # from the dictionary, meta_dict['streams'][0]['tags']['rotate'] is the key
     # we are looking for
     rotateCode = None
     if int(meta_dict['streams'][0]['tags']['rotate']) == 90:
         rotateCode = cv2.ROTATE_90_CLOCKWISE
     elif int(meta_dict['streams'][0]['tags']['rotate']) == 180:
         rotateCode = cv2.ROTATE_180
     elif int(meta_dict['streams'][0]['tags']['rotate']) == 270:
         rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE
         
     return rotateCode

def correct_rotation(frame, rotateCode):  
     return cv2.rotate(frame, rotateCode) 

def norm_inner_feat(X):
  temp_X = X - X[:, :, 0, :].reshape(X.shape[0], X.shape[1], 1, X.shape[-1])
  return temp_X[:, :, 1:, :]


def norm_outer_feat(X, fps=15):
  n_samples = X.shape[0]
  n_frame = X.shape[1]
  n_joint = X.shape[2]
  v = np.zeros((n_samples, n_frame - 1, n_joint, 2))
  time = 1 / fps
  for n in range(n_samples):
      for f in range(n_frame-1):
          v[n,f] = (X[n,f+1] - X[n,f]) / time
  return v
    
ACTION_MAP_ID = {
    0: 'Ngoi', # sit down
    1: 'Dung', # stand up
    2: 'Nhay', # jump
    3: 'Bat tay', # hand shake
    4: 'Di bo', # walk
}

In [ ]:
#for save video visualize
video_path = '/content/drive/MyDrive/dataset/action_videos/IMG_4610.MOV'
showBG = True
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))

try:
  rotateCode = check_rotation(video_path)
except:
  rotateCode = None

if cap.isOpened() is False:
  print('Error opening video stream or file')

fps_time = 0
from google.colab.patches import cv2_imshow
out = cv2.VideoWriter('/content/drive/MyDrive/dataset/demo.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(height), int(width)))

id_1_poses = []
id_2_poses = []
while True:
    ret_val, image = cap.read()
    if rotateCode is not None:
         image = correct_rotation(image, rotateCode)

    try:
      humans = e.inference(image,
                          resize_to_default=(w > 0 and h > 0),
                          upsample_size=4.0)
    except:
      break
    if not showBG:
        image = np.zeros(image.shape)

    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    for idx, ske in enumerate(humans):
      lst_str = str(ske).split('BodyPart:')
      temp_arr = []
      for coor in lst_str:
        if coor:
          key_point = int(coor.split('-')[0])
          if key_point in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
            test_list = coor[coor.find("(") + 1 : coor.find(")")].split(',')
            test_list = [float(i) for i in test_list]
            temp_arr.append(test_list)
      if len(temp_arr) == 14:
        temp_arr = np.array(temp_arr)[[0, 2, 3, 4, 5 ,6 ,7 ,8 ,9 ,10, 11, 12, 13, 1]]
        temp_arr = temp_arr.flatten()
        if idx == 0:
          id_1_poses.append(temp_arr)
        elif idx == 1:
          id_2_poses.append(temp_arr)

    if len(id_1_poses) == 30:
      label_pred = (1-np.asarray(id_1_poses)).reshape(1,30,14,2)
      inner_feat = norm_inner_feat(label_pred)
      outer_feat = norm_outer_feat(label_pred, fps=30)
      X_norm = np.hstack([inner_feat.reshape(len(label_pred), -1), outer_feat.reshape(len(label_pred), -1)])
      label_1 = model.predict(X_norm)
      cv2.putText(image, 'P_1: ' + ACTION_MAP_ID[int(label_1)], (30, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)
      id_1_poses = id_1_poses[1:]


    if len(id_2_poses) == 30:
      label_pred = (1-np.asarray(id_2_poses)).reshape(1,30,14,2)
      inner_feat = norm_inner_feat(label_pred)
      outer_feat = norm_outer_feat(label_pred, fps=30)
      X_norm = np.hstack([inner_feat.reshape(len(label_pred), -1), outer_feat.reshape(len(label_pred), -1)])
      label_2 = model.predict(X_norm)
      cv2.putText(image, 'P_2: ' + ACTION_MAP_ID[int(label_2)], (30, 200),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 3)
      id_2_poses = id_2_poses[1:]

    # cv2_imshow(image)
    out.write(image)
    fps_time = time.time()
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break
out.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
# #for extract csv file


# df = pd.DataFrame(columns=['file_name', 'frame', 'key_point_0', 'key_point_1', 'key_point_2', 'key_point_3', 'key_point_4', 'key_point_5', 'key_point_6', 'key_point_7', 'key_point_8', 'key_point_9', 'key_point_10', 'key_point_11', 'key_point_12', 'key_point_13'])
# for video_path in tqdm(lst_video[:-6]):

#   cap = cv2.VideoCapture(video_path)

#   try:
#     rotateCode = check_rotation(video_path)
#   except:
#     rotateCode = None

#   if cap.isOpened() is False:
#     print('Error opening video stream or file')

#   file_name = os.path.basename(video_path).split('.mp4')[0]
#   coor_arr = []
#   coor_dict = defaultdict(list)
#   frame = 1
#   # frame = frame

#   while True:
#       ret_val, image = cap.read()

#       if rotateCode is not None:
#         image = correct_rotation(image, rotateCode)
#       #run model to extract keypoints of body
#       try:
#         humans = e.inference(image,
#                             resize_to_default=(w > 0 and h > 0),
#                             upsample_size=4.0)
#       except:
#         break
#       #preporcess to get x,y coordinate  
#       if len(humans) >= 1:
#         lst_str = str(humans[0]).split('BodyPart:')
#         temp_arr = []
#         for coor in lst_str:
#           if coor:
#             key_point = int(coor.split('-')[0])
#             if key_point in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
#               temp_arr.append(coor[coor.find("(") + 1 : coor.find(")")].split(','))
#         if len(temp_arr)  < 14:
#           continue
#         else:
#           coor_dict[file_name].append([frame, temp_arr])
#       frame += 1
#   cap.release()

#   #save to csv file
#   len_df = len(df)
#   for key, values in coor_dict.items():
#     for index, val in enumerate(values):
#       df.loc[index + len_df] = [key, val[0], val[1][0], val[1][1], val[1][2], val[1][3], val[1][4], val[1][5], val[1][6], val[1][7], val[1][8], val[1][9], val[1][10], val[1][11], val[1][12], val[1][13]]
# print('Extract data completed !')

In [ ]:
df.to_csv('/content/drive/MyDrive/VinBigdata - PythonBasic/huy_data.csv', index=False)